In [1]:
import os
import re
import random
from typing import List, Dict, Any, Optional
from collections import defaultdict
from time import time
from glob import glob
import numpy as np
import gym
from textworld import EnvInfos
import textworld.gym

#import torch

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
gamefiles = []
GAMES_PATH = "./sample_games/"
for file in os.listdir(GAMES_PATH):
    if file.endswith('.ulx'):
        gamefiles.append(GAMES_PATH + file)

## Play Function
- create gym environment to let the agent play textworld

In [3]:
def play(agent, path, max_step=50, nb_episodes=10, verbose=True):
    request_infos = agent.select_additional_infos
    request_infos.max_score = True  # Needed to normalize the scores.

    gamefiles = [path]
    if os.path.isdir(path):
        gamefiles = glob(os.path.join(path, "*.ulx"))

    env_id = textworld.gym.register_games(gamefiles,
                                          request_infos=request_infos,
                                          max_episode_steps=max_step)
    env = gym.make(env_id)  # Create a Gym environment to play the text game.
    if verbose:
        if os.path.isdir(path):
            print(os.path.dirname(path), end="")
        else:
            print(os.path.basename(path), end="")

    # Collect some statistics: nb_steps, final reward.
    avg_moves, avg_scores, avg_norm_scores = [], [], []
    for no_episode in range(nb_episodes):
        obs, infos = env.reset()  # Start new episode.

        score = 0
        done = False
        nb_moves = 0
        while not done:
            # the agent chooses an action
            command = agent.act(obs, score, done, infos)
            # carry out action and update parameters
            obs, score, done, infos = env.step(command)
            nb_moves += 1

        # Let the agent know the game is done.
        agent.act(obs, score, done, infos)

        if verbose:
            print(".", end="")
        avg_moves.append(nb_moves)
        avg_scores.append(score)
        avg_norm_scores.append(score / infos["max_score"])

    env.close()
    msg = "  \tavg. steps: {:5.1f}; avg. score: {:4.1f} / {}."
    if verbose:
        if os.path.isdir(path):
            print(msg.format(np.mean(avg_moves), np.mean(avg_norm_scores), 1))
        else:
            print(msg.format(np.mean(avg_moves), np.mean(
                avg_scores), infos["max_score"]))

## Custom Agent
- uses Handicap 5 (admissible commands from EnvInfos)
- saves a "qvalue" for each word in vocab.txt
- admissible commands get ranked by the average qvalue of their words
- "qvalues" get updated while the agent is playing depending on scores
    - calculate the reward by subtracting last round's score from this round's score
    - if the command led to a win reward each word an extra +10
    - if the command led to a defeat reward each word an extra -10
    - if the command did not get any reward, add a penalty of -1 (necessary to avoid using the same command over and over again)

In [4]:
class CustomAgent():
    
    def __init__(self) -> None:
        self._initialized = False
        self._epsiode_has_started = False
        self.mode = "test"
        self.command_history = []
        self.history_counter = 0
        
        # read in vocab.txt and map to id
        with open("./vocab.txt") as f:
            self.word_vocab = f.read().split("\n")
        self.word2id = {} #dictionary for converting words to ids
        self.id2word = [] #list for converting ids to words
        for i, w in enumerate(self.word_vocab):
            self.word2id[w] = i
            self.id2word.append(w)
        
        self.qvalues = [0] * len(self.id2word) #rewards for words

    def train(self) -> None:
        self.mode = "train"
        self.transitions = []
        self.last_score = 0
        self.no_train_step = 0
        self.moves = 0
        self.command_history = []
        self.history_counter = 0

    def test(self) -> None:
        self.mode = "test"

    @property
    def select_additional_infos(self) -> EnvInfos:
        """
        Returns what additional information should be made available at each game step.

        Requested information will be included within the `infos` dictionary
        passed to `CustomAgent.act()`. To request specific information, create a
        :py:class:`textworld.EnvInfos <textworld.envs.wrappers.filter.EnvInfos>`
        and set the appropriate attributes to `True`. The possible choices are:

        * `description`: text description of the current room, i.e. output of the `look` command;
        * `inventory`: text listing of the player's inventory, i.e. output of the `inventory` command;
        * `max_score`: maximum reachable score of the game;
        * `objective`: objective of the game described in text;
        * `entities`: names of all entities in the game;
        * `verbs`: verbs understood by the the game;
        * `command_templates`: templates for commands understood by the the game;
        * `admissible_commands`: all commands relevant to the current state;

        In addition to the standard information, game specific information
        can be requested by appending corresponding strings to the `extras`
        attribute. For this competition, the possible extras are:

        * `'recipe'`: description of the cookbook;
        * `'walkthrough'`: one possible solution to the game (not guaranteed to be optimal);

        Example:
            Here is an example of how to request information and retrieve it.

            >>> from textworld import EnvInfos
            >>> request_infos = EnvInfos(description=True, inventory=True, extras=["recipe"])
            ...
            >>> env = gym.make(env_id)
            >>> ob, infos = env.reset()
            >>> print(infos["description"])
            >>> print(infos["inventory"])
            >>> print(infos["extra.recipe"])

            Handicap is defined as follows
                max_score, has_won, has_lost,               # Handicap 0
                description, inventory, verbs, objective,   # Handicap 1
                command_templates,                          # Handicap 2
                entities,                                   # Handicap 3
                extras=["recipe"],                          # Handicap 4
                admissible_commands,                        # Handicap 5
        """
        return EnvInfos(description=True, inventory=True, max_score = True, 
                        admissible_commands=True, has_won=True, has_lost=True)

    def _init(self) -> None:
        """ Initialize the agent. """
        self._initialized = True

        # [You can insert code here.]
    
    def _get_word_id(self, word):
        """
        look up id of a word in dictionary & add word to dictionaries if unknown.
        """
        if word not in self.word2id:        
            self.id2word.append(word)
            self.qvalues.append(0)
            self.word2id[word] = len(self.word2id)
        return self.word2id[word]

    
    def _get_word_by_id(self, id):
        """
        look up word by id in dictionary.
        returns None, if id does not exist.
        """
        if id < len(self.id2word):
            return self.id2word[id]
        else:
            return None
    
    def _get_qvalue(self, id):
        """
        Getter for qvalues (words with scores).
        """
        if id < len(self.qvalues):
            return self.qvalues[id]
        else:
            return None
    
    def _tokenize(self, text):
        """
        Simple tokenizer: strip out all non-alphabetic characters.
        """
        text = re.sub("[^a-zA-Z0-9\- ]", " ", text)
        word_ids = list(map(self._get_word_id, text.split()))
        return word_ids
    
    def _start_episode(self, obs: List[str], infos: Dict[str, List[Any]]) -> None:
        """
        Prepare the agent for the upcoming episode.
        Arguments:
            obs: Initial feedback for each game.
            infos: Additional information for each game.
        """
        if not self._initialized:
            self._init()

        self._epsiode_has_started = True

        # [You can insert code here.]

    def _end_episode(self, obs: List[str], scores: List[int], infos: Dict[str, List[Any]]) -> None:
        """
        Tell the agent the episode has terminated.
        Arguments:
            obs: Previous command's feedback for each game.
            score: The score obtained so far for each game.
            infos: Additional information for each game.
        """
        self._epsiode_has_started = False

        # [You can insert code here.]
    
    def update_qvalues(self, command : List[int],reward):
        """
        Updates words with additional rewards.
        """
        for id in command:
            self.qvalues[id] += reward
    
    def calc_avg_qvalue(self, command) -> int:
        """
        Gets the average of all qvalues of words in a command as a sum.
        """
        sum = 0
        for id in command:
            sum += self._get_qvalue(id)
        sum = sum/ len(command)
        return sum

    def choose_best_cmd(self,commands) -> int:
        """
        Gets randomly one of the best rewarded admissible commands.
        """
        avg_qvalues =  []
        for cmd in commands:
            avg_qvalues.append(self.calc_avg_qvalue(cmd))
        max_value = max(avg_qvalues)
        max_commands = []
        for i in range(len(commands)):
            if(max_value == avg_qvalues[i]):
                max_commands.append(i)
        return random.choice(max_commands)
    
    def get_alternative_cmd(self, commands) -> int:
        """
        Idea of an alternative command, if the current is used in the last 3 commands.
        Chooses the second best command.
        """
        avg_qvalues =  []
        for cmd in commands:
            avg_qvalues.append(self.calc_avg_qvalue(cmd))
     
        first_max = max(avg_qvalues[0],avg_qvalues[1]) 
        second_max = min(avg_qvalues[0],avg_qvalues[1]) 

        for i in range(2,len(avg_qvalues)): 
            if avg_qvalues[i] > first_max: 
                second_max = first_max
                first_max=avg_qvalues[i] 
            else: 
                if avg_qvalues[i]>second_max: 
                    second_max=avg_qvalues[i] 

        print("Second highest number is : ",str(second_max)) 
        return avg_qvalues.index(second_max)
        
                        
    def act(self, obs: str, score: int, done: bool, 
            infos: Dict[str, List[Any]]) -> Optional[List[str]]:
        """
        Acts upon the current list of observations.

        One text command must be returned for each observation.

        Arguments:
            obs: Previous command's feedback for each game.
            scores: The score obtained so far for each game.
            dones: Whether a game is finished.
            infos: Additional information for each game.

        Returns:
            Text commands to be performed (one per observation).
            If episode had ended (e.g. `all(dones)`), the returned
            value is ignored.

        Notes:
            Commands returned for games marked as `done` have no effect.
            The states for finished games are simply copy over until all
            games are done.
        """
        print("obs in act: ", obs)
        #if all(dones):
        #   self._end_episode(obs, scores, infos)
        #    return  # Nothing to return.

        if not self._epsiode_has_started:
            self._start_episode(obs, infos)
        
        #choose best command based on "qvalues"
        cmds = []
        for i in range(len(infos["admissible_commands"])):
            cmds.append(self._tokenize(infos["admissible_commands"][i]))
        best_index = self.choose_best_cmd(cmds)
        action = infos["admissible_commands"][best_index]
        
        #building a historylist of commands to get out of a loop of death o_o
        if self.command_history is not None:
            last_commands = self.command_history[-3:]
            
        # compare-counters for comparing actions from the 3 last moves.
        if self.history_counter > self.moves:
            same_command_in_list = True 
            counter_use_same_command = 0
            """ 
            If the current admissible command (action) is one of the last 3 commands, the while-loop
            will be skipped. 
            If the same random chosen command is going 2 times through the while-loop, after tahat, the loops
            will be skipped, too.
            """
            while same_command_in_list is True and counter_use_same_command < 3:
                try:
                    if last_commands.index(action):
                        #best_index = self.choose_best_cmd(cmds)
                        best_index = self.get_alternative_cmd(cmds)
                        action = infos["admissible_commands"][best_index]
                        counter_use_same_command += 1
                    else:
                        same_command_in_list = False
                except ValueError:
                    same_command_in_list = False
                    print("last_commands List does not contain value")
        # adds valid action to the command-history
        self.command_history.append(action)
        
        if self.mode == "test":
            return action
        
        #train mode, counter update
        self.no_train_step += 1 
        reward = 0
        self.moves += 1
        self.history_counter += 1
        
        #calculate rewards
        if self.transitions:
            reward = score - self.last_score
            self.last_score = score
            if infos["has_won"]:
                reward += 10
            if infos["has_lost"]:
                reward -= 10
            if reward == 0:
                reward -= 1
            # update rewards for command from last step
            self.update_qvalues(self.transitions[-1],reward)

        # Debug output
        if(self.no_train_step % 1 == 0):
            print("")
            print("Last 3 admissible commands: ", last_commands)
            print("train step:" + str(self.no_train_step))
            print("last reward:" + str(reward))
            command = "last command: "
            if self.transitions:
                for id in self.transitions[-1]:
                    command += " " + self._get_word_by_id(id)
                print(command)
        
        #save last command in order to calculate rewards in next step
        self.transitions.append(cmds[best_index])
        
        if done:
            if(infos["has_won"]):
                print("-------- WON GAME ----------")
            else:
                print("-------- LOST GAME ----------")
            print(" ")
            self.last_score = 0
            self.moves = 0
            self.transitions = []
            self.command_history = []
            
        return action

## Training the Agent
- just training on the easiest game

In [5]:
agent = CustomAgent()
agent.train()
starttime = time()
for i in range(5):
    print("-------------------------------------")
    print("GAME: " + str(i+1))
    print("-------------------------------------")
    play(agent, gamefiles[5])
print("Trained in {:.2f} secs".format(time() - starttime))


-------------------------------------
GAME: 1
-------------------------------------


/home/noy/venv3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


tw-cooking-recipe1+take1-11Oeig8bSVdGSp78.ulxobs in act:  


                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
                      | $$   | $$__     \$$\/  $$   | $$          
                      | $$   | $$  \     >$$  $$    | $$          
                      | $$   | $$$$$    /  $$$$\    | $$          
                      | $$   | $$_____ |  $$ \$$\   | $$          
                      | $$   | $$     \| $$  | $$   | $$          
                       \$$    \$$$$$$$$ \$$   \$$    \$$          
              __       __   ______   _______   __        _______  
             |  \  _  |  \ /      \ |       \ |  \      |       \ 
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$  | $$
 

obs in act:  You take the red hot pepper from the counter.



Your score has just gone up by one point.

last_commands List does not contain value

Last 3 admissible commands:  ['take red hot pepper from counter']
train step:28
last reward:1
last command:  take red hot pepper from counter
obs in act:  Adding the meal to your inventory.



Your score has just gone up by one point.

last_commands List does not contain value

Last 3 admissible commands:  ['take red hot pepper from counter', 'prepare meal']
train step:29
last reward:1
last command:  prepare meal
obs in act:  You eat the meal. Not bad.


Your score has just gone up by one point.


                               *** The End ***

You scored 3 out of a possible 3, in 5 turn(s).


Would you like to QUIT?
> 
last_commands List does not contain value

Last 3 admissible commands:  ['take red hot pepper from counter', 'prepare meal', 'eat meal']
train step:30
last reward:11
last command:  eat meal
-------- WON GAME ----------
 
.ob

obs in act:  You eat the meal. Not bad.


Your score has just gone up by one point.


                               *** The End ***

You scored 3 out of a possible 3, in 5 turn(s).


Would you like to QUIT?
> 
last_commands List does not contain value

Last 3 admissible commands:  ['take red hot pepper from counter', 'prepare meal', 'eat meal']
train step:46
last reward:11
last command:  eat meal
-------- WON GAME ----------
 
.obs in act:  


                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
                      | $$   | $$__     \$$\/  $$   | $$          
                      | $$   | $$  \     >$$  $$    | $$          
                      | $$   | $$$$$    /  $$$$\    | $$          
                      | $$   | $$_____ |  $$ \$$\   | $$          
                      | $$   | $$     \| $$  | $$   | $$          
               

...

obs in act:  Adding the meal to your inventory.



Your score has just gone up by one point.

last_commands List does not contain value

Last 3 admissible commands:  ['take red hot pepper from counter', 'prepare meal']
train step:157
last reward:1
last command:  prepare meal
obs in act:  You eat the meal. Not bad.


Your score has just gone up by one point.


                               *** The End ***

You scored 3 out of a possible 3, in 5 turn(s).


Would you like to QUIT?
> 
last_commands List does not contain value

Last 3 admissible commands:  ['take red hot pepper from counter', 'prepare meal', 'eat meal']
train step:158
last reward:11
last command:  eat meal
-------- WON GAME ----------
 
.obs in act:  


                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
                      | $$   | $$__     \$$\/  $$   | $$          
      

.obs in act:  


                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
                      | $$   | $$__     \$$\/  $$   | $$          
                      | $$   | $$  \     >$$  $$    | $$          
                      | $$   | $$$$$    /  $$$$\    | $$          
                      | $$   | $$_____ |  $$ \$$\   | $$          
                      | $$   | $$     \| $$  | $$   | $$          
                       \$$    \$$$$$$$$ \$$   \$$    \$$          
              __       __   ______   _______   __        _______  
             |  \  _  |  \ /      \ |       \ |  \      |       \ 
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$  | $$
             | $$ $$\$$\$$| $$  | $$| $$$$$$$

.obs in act:  


                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
                      | $$   | $$__     \$$\/  $$   | $$          
                      | $$   | $$  \     >$$  $$    | $$          
                      | $$   | $$$$$    /  $$$$\    | $$          
                      | $$   | $$_____ |  $$ \$$\   | $$          
                      | $$   | $$     \| $$  | $$   | $$          
                       \$$    \$$$$$$$$ \$$   \$$    \$$          
              __       __   ______   _______   __        _______  
             |  \  _  |  \ /      \ |       \ |  \      |       \ 
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$  | $$
             | $$ $$\$$\$$| $$  | $$| $$$$$$$

obs in act:  You take the red hot pepper from the counter.



Your score has just gone up by one point.

last_commands List does not contain value

Last 3 admissible commands:  ['take red hot pepper from counter']
train step:200
last reward:1
last command:  take red hot pepper from counter
obs in act:  Adding the meal to your inventory.



Your score has just gone up by one point.

last_commands List does not contain value

Last 3 admissible commands:  ['take red hot pepper from counter', 'prepare meal']
train step:201
last reward:1
last command:  prepare meal
obs in act:  You eat the meal. Not bad.


Your score has just gone up by one point.


                               *** The End ***

You scored 3 out of a possible 3, in 5 turn(s).


Would you like to QUIT?
> 
last_commands List does not contain value

Last 3 admissible commands:  ['take red hot pepper from counter', 'prepare meal', 'eat meal']
train step:202
last reward:11
last command:  eat meal
-------- WON GAME ----------
 


obs in act:  Adding the meal to your inventory.



Your score has just gone up by one point.

last_commands List does not contain value

Last 3 admissible commands:  ['take red hot pepper from counter', 'prepare meal']
train step:217
last reward:1
last command:  prepare meal
obs in act:  You eat the meal. Not bad.


Your score has just gone up by one point.


                               *** The End ***

You scored 3 out of a possible 3, in 5 turn(s).


Would you like to QUIT?
> 
last_commands List does not contain value

Last 3 admissible commands:  ['take red hot pepper from counter', 'prepare meal', 'eat meal']
train step:218
last reward:11
last command:  eat meal
-------- WON GAME ----------
 
.  	avg. steps:   3.0; avg. score:  3.0 / 3.
Trained in 7.53 secs


In [94]:
agent.test()
play(agent, gamefiles[5])

tw-cooking-recipe3+take3+cook+cut+drop-vNZGCrQVhoJdt7Nx.ulxobs in act:  


                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
                      | $$   | $$__     \$$\/  $$   | $$          
                      | $$   | $$  \     >$$  $$    | $$          
                      | $$   | $$$$$    /  $$$$\    | $$          
                      | $$   | $$_____ |  $$ \$$\   | $$          
                      | $$   | $$     \| $$  | $$   | $$          
                       \$$    \$$$$$$$$ \$$   \$$    \$$          
              __       __   ______   _______   __        _______  
             |  \  _  |  \ /      \ |       \ |  \      |       \ 
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$    

obs in act:  You open the fridge, revealing a red onion, a raw pork chop, a red hot pepper, a white onion, a carrot and an orange bell pepper.


obs in act:  You close the fridge.


Second highest number is :  -234.0
obs in act:  -= Kitchen =-
You've seen better kitchens, but at least this one seems pretty ordinary.

You see a fridge. You can see a closed oven. You make out a table. The table is massive. But the thing is empty, unfortunately. Hm. Oh well You can see a counter. The counter is vast. On the counter you can see a red apple, a raw yellow potato, a raw red potato, a yellow bell pepper and a raw purple potato. Classic TextWorld. You rest your hand against a wall, but you miss the wall and fall onto a stove. The stove is conventional. However, the stove, like an empty stove, has nothing on it. Sometimes, just sometimes, TextWorld can just be the worst.



There is a knife and a cookbook on the floor.


obs in act:  You open the fridge, revealing a red onion, a raw pork chop, a

obs in act:  You open the fridge, revealing a red onion, a raw pork chop, a red hot pepper, a white onion, a carrot and an orange bell pepper.


Second highest number is :  -234.0
Second highest number is :  -234.0
Second highest number is :  -234.0
obs in act:  You close the fridge.


Second highest number is :  -234.0
last_commands List does not contain value
obs in act:  -= Kitchen =-
You've seen better kitchens, but at least this one seems pretty ordinary.

You see a fridge. You can see a closed oven. You make out a table. The table is massive. But the thing is empty, unfortunately. Hm. Oh well You can see a counter. The counter is vast. On the counter you can see a red apple, a raw yellow potato, a raw red potato, a yellow bell pepper and a raw purple potato. Classic TextWorld. You rest your hand against a wall, but you miss the wall and fall onto a stove. The stove is conventional. However, the stove, like an empty stove, has nothing on it. Sometimes, just sometimes, TextWorld ca

obs in act:  You close the fridge.


Second highest number is :  -234.0
obs in act:  -= Kitchen =-
You've seen better kitchens, but at least this one seems pretty ordinary.

You see a fridge. You can see a closed oven. You make out a table. The table is massive. But the thing is empty, unfortunately. Hm. Oh well You can see a counter. The counter is vast. On the counter you can see a red apple, a raw yellow potato, a raw red potato, a yellow bell pepper and a raw purple potato. Classic TextWorld. You rest your hand against a wall, but you miss the wall and fall onto a stove. The stove is conventional. However, the stove, like an empty stove, has nothing on it. Sometimes, just sometimes, TextWorld can just be the worst.



There is a knife and a cookbook on the floor.


.obs in act:  


                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
  

obs in act:  You close the fridge.


Second highest number is :  -234.0
obs in act:  -= Kitchen =-
You've seen better kitchens, but at least this one seems pretty ordinary.

You see a fridge. You can see a closed oven. You make out a table. The table is massive. But the thing is empty, unfortunately. Hm. Oh well You can see a counter. The counter is vast. On the counter you can see a red apple, a raw yellow potato, a raw red potato, a yellow bell pepper and a raw purple potato. Classic TextWorld. You rest your hand against a wall, but you miss the wall and fall onto a stove. The stove is conventional. However, the stove, like an empty stove, has nothing on it. Sometimes, just sometimes, TextWorld can just be the worst.



There is a knife and a cookbook on the floor.


obs in act:  You open the fridge, revealing a red onion, a raw pork chop, a red hot pepper, a white onion, a carrot and an orange bell pepper.


obs in act:  You close the fridge.


Second highest number is :  -234.0
obs

...

obs in act:  You close the fridge.


Second highest number is :  -234.0
obs in act:  -= Kitchen =-
You've seen better kitchens, but at least this one seems pretty ordinary.

You see a fridge. You can see a closed oven. You make out a table. The table is massive. But the thing is empty, unfortunately. Hm. Oh well You can see a counter. The counter is vast. On the counter you can see a red apple, a raw yellow potato, a raw red potato, a yellow bell pepper and a raw purple potato. Classic TextWorld. You rest your hand against a wall, but you miss the wall and fall onto a stove. The stove is conventional. However, the stove, like an empty stove, has nothing on it. Sometimes, just sometimes, TextWorld can just be the worst.



There is a knife and a cookbook on the floor.


obs in act:  You open the fridge, revealing a red onion, a raw pork chop, a red hot pepper, a white onion, a carrot and an orange bell pepper.


obs in act:  You close the fridge.


Second highest number is :  -234.0
obs

obs in act:  -= Kitchen =-
You've seen better kitchens, but at least this one seems pretty ordinary.

You see a fridge. You can see a closed oven. You make out a table. The table is massive. But the thing is empty, unfortunately. Hm. Oh well You can see a counter. The counter is vast. On the counter you can see a red apple, a raw yellow potato, a raw red potato, a yellow bell pepper and a raw purple potato. Classic TextWorld. You rest your hand against a wall, but you miss the wall and fall onto a stove. The stove is conventional. However, the stove, like an empty stove, has nothing on it. Sometimes, just sometimes, TextWorld can just be the worst.



There is a knife and a cookbook on the floor.


obs in act:  You open the fridge, revealing a red onion, a raw pork chop, a red hot pepper, a white onion, a carrot and an orange bell pepper.


Second highest number is :  -234.0
obs in act:  You close the fridge.


Second highest number is :  -234.0
obs in act:  -= Kitchen =-
You've seen b

obs in act:  You drop the knife on the ground.


last_commands List does not contain value
obs in act:  You close the fridge.


last_commands List does not contain value
obs in act:  You open the fridge, revealing a red onion, a raw pork chop, a red hot pepper, a white onion, a carrot and an orange bell pepper.


last_commands List does not contain value
obs in act:  -= Kitchen =-
You've seen better kitchens, but at least this one seems pretty ordinary.

You see a fridge. The fridge contains a red onion, a raw pork chop, a red hot pepper, a white onion, a carrot and an orange bell pepper. You can see a closed oven. You make out a table. The table is massive. But the thing is empty, unfortunately. Hm. Oh well You can see a counter. The counter is vast. On the counter you can see a red apple, a raw yellow potato, a raw red potato, a yellow bell pepper and a raw purple potato. Classic TextWorld. You rest your hand against a wall, but you miss the wall and fall onto a stove. The stove is c

obs in act:  You close the fridge.


Second highest number is :  -234.0
obs in act:  -= Kitchen =-
You've seen better kitchens, but at least this one seems pretty ordinary.

You see a fridge. You can see a closed oven. You make out a table. The table is massive. But the thing is empty, unfortunately. Hm. Oh well You can see a counter. The counter is vast. On the counter you can see a red apple, a raw yellow potato, a raw red potato, a yellow bell pepper and a raw purple potato. Classic TextWorld. You rest your hand against a wall, but you miss the wall and fall onto a stove. The stove is conventional. However, the stove, like an empty stove, has nothing on it. Sometimes, just sometimes, TextWorld can just be the worst.



There is a knife and a cookbook on the floor.


obs in act:  You open the fridge, revealing a red onion, a raw pork chop, a red hot pepper, a white onion, a carrot and an orange bell pepper.


Second highest number is :  -234.0
obs in act:  You close the fridge.


Sec

obs in act:  -= Kitchen =-
You've seen better kitchens, but at least this one seems pretty ordinary.

You see a fridge. You can see a closed oven. You make out a table. The table is massive. But the thing is empty, unfortunately. Hm. Oh well You can see a counter. The counter is vast. On the counter you can see a red apple, a raw yellow potato, a raw red potato, a yellow bell pepper and a raw purple potato. Classic TextWorld. You rest your hand against a wall, but you miss the wall and fall onto a stove. The stove is conventional. However, the stove, like an empty stove, has nothing on it. Sometimes, just sometimes, TextWorld can just be the worst.



There is a knife and a cookbook on the floor.


obs in act:  You open the fridge, revealing a red onion, a raw pork chop, a red hot pepper, a white onion, a carrot and an orange bell pepper.


Second highest number is :  -234.0
obs in act:  You close the fridge.


Second highest number is :  -234.0
obs in act:  -= Kitchen =-
You've seen b

In [95]:
for i in range(len(agent.qvalues)):
    if(agent.qvalues[i] != 0 ):
        print(agent._get_word_by_id(i) + " : " + str(agent.qvalues[i]))

apple : -189
banana : -114
bell : -209
carrot : -46
chop : -140
close : -135
cook : -155
cookbook : -141
counter : -150
drop : -142
eat : -120
examine : -424
fridge : -333
from : -283
hot : -42
insert : -62
into : -62
knife : -142
look : -234
meal : -59
on : -155
onion : -140
open : -134
orange : -57
oven : -146
pepper : -251
pork : -140
potato : -225
prepare : -59
purple : -188
put : -155
red : -232
stove : -148
table : -148
take : -331
white : -140
with : -155
yellow : -188


In [96]:
agent.train()
starttime = time()
for i in range(5):
    print("-------------------------------------")
    print("GAME: " + str(i+1))
    print("-------------------------------------")
    play(agent, gamefiles[8])
print("Trained in {:.2f} secs".format(time() - starttime))

-------------------------------------
GAME: 1
-------------------------------------
tw-cooking-recipe3+cook+cut+drop+go12-B10qS58OTbe0T8Qk.ulxobs in act:  


                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
                      | $$   | $$__     \$$\/  $$   | $$          
                      | $$   | $$  \     >$$  $$    | $$          
                      | $$   | $$$$$    /  $$$$\    | $$          
                      | $$   | $$_____ |  $$ \$$\   | $$          
                      | $$   | $$     \| $$  | $$   | $$          
                       \$$    \$$$$$$$$ \$$   \$$    \$$          
              __       __   ______   _______   __        _______  
             |  \  _  |  \ /      \ |       \ |  \      |       \ 
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| 

obs in act:  
-= Backyard =-
You have stumbled into the most usual of all possible backyards.

You make out a patio table. What a coincidence, weren't you just thinking about a patio table? But there isn't a thing on it. You bend down to tie your shoe. When you stand up, you notice a patio chair. What a coincidence, weren't you just thinking about a patio chair? Looks like someone's already been here and taken everything off it, though. What, you think everything in TextWorld should have stuff on it? You can see a closed BBQ.

There is an open barn door leading east. There is an open sliding patio door leading south. There is an exit to the north. Don't worry, there is no door.




Last 3 admissible commands:  ['go south', 'go east', 'go west']
train step:22
last reward:-1
last command:  go west
obs in act:  
-= Garden =-
Well, here we are in a garden.



There is an exit to the south.




Last 3 admissible commands:  ['go east', 'go west', 'go north']
train step:23
last reward:-1
last

obs in act:  
-= Garden =-
Well, here we are in a garden.



There is an exit to the south.




Last 3 admissible commands:  ['go east', 'go west', 'go north']
train step:37
last reward:-1
last command:  go north
obs in act:  
-= Backyard =-
You have stumbled into the most usual of all possible backyards.

You make out a patio table. What a coincidence, weren't you just thinking about a patio table? But there isn't a thing on it. You bend down to tie your shoe. When you stand up, you notice a patio chair. What a coincidence, weren't you just thinking about a patio chair? Looks like someone's already been here and taken everything off it, though. What, you think everything in TextWorld should have stuff on it? You can see a closed BBQ.

There is an open barn door leading east. There is an open sliding patio door leading south. There is an exit to the north. Don't worry, there is no door.




Last 3 admissible commands:  ['go west', 'go north', 'go south']
train step:38
last reward:-1
la

obs in act:  
-= Backyard =-
You have stumbled into the most usual of all possible backyards.

You make out a patio table. What a coincidence, weren't you just thinking about a patio table? But there isn't a thing on it. You bend down to tie your shoe. When you stand up, you notice a patio chair. What a coincidence, weren't you just thinking about a patio chair? Looks like someone's already been here and taken everything off it, though. What, you think everything in TextWorld should have stuff on it? You can see a closed BBQ.

There is an open barn door leading east. There is a closed sliding patio door leading south. There is an exit to the north. Don't worry, there is no door.



last_commands List does not contain value

Last 3 admissible commands:  ['close sliding patio door', 'go north', 'go south']
train step:69
last reward:-1
last command:  go south
obs in act:  
-= Shed =-
Look at you, bigshot, walking into a shed like it isn't some huge deal.

You make out a toolbox. The toolb

.obs in act:  


                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
                      | $$   | $$__     \$$\/  $$   | $$          
                      | $$   | $$  \     >$$  $$    | $$          
                      | $$   | $$$$$    /  $$$$\    | $$          
                      | $$   | $$_____ |  $$ \$$\   | $$          
                      | $$   | $$     \| $$  | $$   | $$          
                       \$$    \$$$$$$$$ \$$   \$$    \$$          
              __       __   ______   _______   __        _______  
             |  \  _  |  \ /      \ |       \ |  \      |       \ 
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$  | $$
             | $$ $$\$$\$$| $$  | $$| $$$$$$$

...

obs in act:  -= Garden =-
Well, here we are in a garden.



There is an exit to the south.

There is a red hot pepper, a red onion and a salt on the floor.


Second highest number is :  -322.0
last_commands List does not contain value

Last 3 admissible commands:  ['take red hot pepper', 'take red hot pepper', 'look']
train step:1597
last reward:-1
last command:  look
obs in act:  
-= Backyard =-
You have stumbled into the most usual of all possible backyards.

You make out a patio table. What a coincidence, weren't you just thinking about a patio table? But there isn't a thing on it. You bend down to tie your shoe. When you stand up, you notice a patio chair. What a coincidence, weren't you just thinking about a patio chair? Looks like someone's already been here and taken everything off it, though. What, you think everything in TextWorld should have stuff on it? You can see a closed BBQ.

There is an open barn door leading east. There is an open sliding patio door leading south. Ther

obs in act:  You open sliding patio door.


last_commands List does not contain value

Last 3 admissible commands:  ['examine patio chair', 'close sliding patio door', 'open sliding patio door']
train step:1639
last reward:-1
last command:  open sliding patio door
obs in act:  
-= Garden =-
Well, here we are in a garden.



There is an exit to the south.

There is a red hot pepper, a red onion and a salt on the floor.


last_commands List does not contain value

Last 3 admissible commands:  ['close sliding patio door', 'open sliding patio door', 'go north']
train step:1640
last reward:-1
last command:  go north
obs in act:  -= Garden =-
Well, here we are in a garden.



There is an exit to the south.

There is a red hot pepper, a red onion and a salt on the floor.


Second highest number is :  -325.0
last_commands List does not contain value

Last 3 admissible commands:  ['open sliding patio door', 'go north', 'look']
train step:1641
last reward:-1
last command:  look
obs in act:  You'

.obs in act:  


                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
                      | $$   | $$__     \$$\/  $$   | $$          
                      | $$   | $$  \     >$$  $$    | $$          
                      | $$   | $$$$$    /  $$$$\    | $$          
                      | $$   | $$_____ |  $$ \$$\   | $$          
                      | $$   | $$     \| $$  | $$   | $$          
                       \$$    \$$$$$$$$ \$$   \$$    \$$          
              __       __   ______   _______   __        _______  
             |  \  _  |  \ /      \ |       \ |  \      |       \ 
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$  | $$
             | $$ $$\$$\$$| $$  | $$| $$$$$$$

obs in act:  -= Garden =-
Well, here we are in a garden.



There is an exit to the south.

There is a red hot pepper, a red onion and a salt on the floor.


Second highest number is :  -332.5
last_commands List does not contain value

Last 3 admissible commands:  ['take red onion', 'take red onion', 'look']
train step:1688
last reward:-1
last command:  look
obs in act:  
-= Backyard =-
You have stumbled into the most usual of all possible backyards.

You make out a patio table. What a coincidence, weren't you just thinking about a patio table? But there isn't a thing on it. You bend down to tie your shoe. When you stand up, you notice a patio chair. What a coincidence, weren't you just thinking about a patio chair? Looks like someone's already been here and taken everything off it, though. What, you think everything in TextWorld should have stuff on it? You can see a closed BBQ.

There is an open barn door leading east. There is an open sliding patio door leading south. There is an ex

obs in act:  You open sliding patio door.


last_commands List does not contain value

Last 3 admissible commands:  ['go west', 'close sliding patio door', 'open sliding patio door']
train step:1713
last reward:-1
last command:  open sliding patio door
obs in act:  
-= Garden =-
Well, here we are in a garden.



There is an exit to the south.

There is a red hot pepper, a red onion and a salt on the floor.


last_commands List does not contain value

Last 3 admissible commands:  ['close sliding patio door', 'open sliding patio door', 'go north']
train step:1714
last reward:-1
last command:  go north
obs in act:  -= Garden =-
Well, here we are in a garden.



There is an exit to the south.

There is a red hot pepper, a red onion and a salt on the floor.


Second highest number is :  -334.25
last_commands List does not contain value

Last 3 admissible commands:  ['open sliding patio door', 'go north', 'look']
train step:1715
last reward:-1
last command:  look
obs in act:  You're carrying

In [44]:
# versuche
wordids = agent._tokenize("cook a meal")
print(wordids)

print(agent._get_word_id("cook"))
print(agent._get_word_by_id(4664))

[4664, 785, 11449]
4664
cook


In [78]:
def play_render(agent,gamefile):
    requested_infos = agent.select_additional_infos
    env_id = textworld.gym.register_games([gamefile], requested_infos)

    env = gym.make(env_id)
    obs, infos = env.reset()

    env.render()  # Print the initial observation.

    score = 0
    done = False
    while not done:
        command = agent.act(obs,score,done,infos)
        ob, score, done, infos = env.step(command)
        env.render()

In [79]:
agent.test()
play_render(agent, gamefiles[5])




                    ________  ________  __    __  ________
                   |        \|        \|  \  |  \|        \
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$
                      | $$   | $$__     \$$\/  $$   | $$
                      | $$   | $$  \     >$$  $$    | $$
                      | $$   | $$$$$    /  $$$$\    | $$
                      | $$   | $$_____ |  $$ \$$\   | $$
                      | $$   | $$     \| $$  | $$   | $$
                       \$$    \$$$$$$$$ \$$   \$$    \$$
              __       __   ______   _______   __        _______
             |  \  _  |  \ /      \ |       \ |  \      |       \
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$  | $$
             | $$ $$\$$\$$| $$  | $$| $$$$$$$\| $$      | $$  | $$
             | $$$$  \$$$$| $$__/ $$| $$  | $$| $$_____ | $$__/ $$
          